In [3]:
# import dash
# from dash import html, Input, Output

# app = dash.Dash(__name__)


# toggler_on = {'background-color': 'black',
#                'color': 'white',
#                'height': '30px',
#                'width': '100px',
#                'margin-top': '20px',  # Reduced padding
#                'margin-left': '20px',  # Reduced padding
#                'margin-right': '-15px',
#                'border-radius': '10px',  # Smooth edges
#                'box-shadow': '0px 4px 8px rgba(0, 0, 0, 0.2)',  # Floating effect
#                'border': 'none',  # Remove border               
#                'position': 'relative',
#                'z-index': '1'}

# toggler_off = {'background-color': 'white',
#                'color': 'black',
#                'height': '30px',
#                'width': '100px',
#                'margin-top': '20px',  # Reduced padding
#                'margin-left': '20px',  # Reduced padding
#                'margin-right': '-15px',
#                'border-radius': '10px',  # Smooth edges
#                'box-shadow': '0px 4px 8px rgba(0, 0, 0, 0.2)',  # Floating effect
#                'border': 'none',  # Remove border               
#                'position': 'relative',
#                'z-index': '1'}

# button_2024 = {'background-color': '#c00000',
#                'color': 'white',
#                'height': '30px',
#                'width': '50px',
#                'margin-top': '20px',  # Reduced padding
#                'margin-left': '20px',  # Reduced padding
#                'margin-right': '-15px',
#                'border-radius': '10px',  # Smooth edges
#                'box-shadow': '0px 4px 8px rgba(0, 0, 0, 0.2)',  # Floating effect
#                'border': 'none',  # Remove border               
#                'position': 'relative',
#                'z-index': '1'}

# button_2023 = {'background-color': '#e97132',
#                'color': 'white',
#                'height': '30px',
#                'width': '50px',
#                'margin-top': '20px',  # Reduced padding
#                'margin-left': '20px',  # Reduced padding
#                'margin-right': '-15px',               
#                'border-radius': '10px',  # Smooth edges
#                'box-shadow': '0px 4px 8px rgba(0, 0, 0, 0.2)',  # Floating effect
#                'border': 'none',  # Remove border               
#                'position': 'relative',
#                'z-index': '1'}

# button_2022 = {'background-color': '#bfbec4',
#                'color': 'white',
#                'height': '30px',
#                'width': '50px',
#                'margin-top': '20px',  # Reduced padding
#                'margin-left': '20px',  # Reduced padding
#                'border-radius': '10px',  # Smooth edges
#                'box-shadow': '0px 4px 8px rgba(0, 0, 0, 0.2)',  # Floating effect
#                'border': 'none',  # Remove border               
#                'position': 'relative',
#                'z-index': '1'}

# off_button = {'background-color': 'white',
#               'color': 'grey',
#               'height': '30px',
#               'width': '50px',
#               'margin-top': '20px',  # Reduced padding
#               'margin-left': '20px',  # Reduced padding
#                'margin-right': '-15px',                             
#               'border-radius': '10px',  # Smooth edges
#               'box-shadow': '0px 4px 8px rgba(0, 0, 0, 0.2)',  # Floating effect
#                'border': 'none',  # Remove border              
#               'position': 'relative',
#               'z-index': '1'}

# app.layout = html.Div([
    
    
#     html.Button(id='chart_toggle',
#                 children=['chart_toggle'],
#                 n_clicks=0,
#                 style=toggler_on
#     ),
#     html.Button(id='range_toggle',
#                 children=['range_toggle'],
#                 n_clicks=0,
#                 style=toggler_on
#     ),
    
#     html.Button(id='2024_toggle',
#                 children=['2024'],
#                 n_clicks=0,
#                 style=button_2024
#     ),
#     html.Button(id='2023_toggle',
#                 children=['2023'],
#                 n_clicks=0,
#                 style=button_2023
#     ),
#     html.Button(id='2022_toggle',
#                 children=['2022'],
#                 n_clicks=0,
#                 style=button_2022
#     ),
# ])

# @app.callback(
#     Output('chart_toggle', 'style'), 
#     Output('chart_toggle', 'children'),     
#     [Input('chart_toggle', 'n_clicks')]
# )
# def change_chart_button_style(n_clicks):
#     if n_clicks % 2 == 1:
#         return toggler_off, 'Line'
#     else:
#         return toggler_on, 'Seasonality'

# @app.callback(
#     Output('range_toggle', 'style'), 
#     Output('range_toggle', 'children'),     
#     [Input('range_toggle', 'n_clicks')]
# )
# def change_chart_button_style(n_clicks):
#     if n_clicks % 2 == 1:
#         return toggler_off, '5 year'
#     else:
#         return toggler_on, '2015-2019'

# @app.callback(
#     Output('2024_toggle', 'style'), 
#     [Input('2024_toggle', 'n_clicks')]
# )
# def change_2024_button_style(n_clicks):
#     if n_clicks % 2 == 1:
#         return off_button
#     else:
#         return button_2024

# @app.callback(
#     Output('2023_toggle', 'style'), 
#     [Input('2023_toggle', 'n_clicks')]
# )
# def change_2023_button_style(n_clicks):
#     if n_clicks % 2 == 1:
#         return off_button
#     else:
#         return button_2023

# @app.callback(
#     Output('2022_toggle', 'style'), 
#     [Input('2022_toggle', 'n_clicks')]
# )
# def change_2022_button_style(n_clicks):
#     if n_clicks % 2 == 1:
#         return off_button
#     else:
#         return button_2022

# if __name__ == '__main__':
#     app.run_server(debug=True,port=8051)


In [38]:
import pandas as pd
df = pd.read_csv('./data/wps_gte_2015_pivot.csv',parse_dates=['period'])
df = df.iloc[:-1,:]
df.to_csv('./data/wps_gte_2015_pivot.csv',index=False)
df

,period,WCEIMP12,WCEIMP22,WCEIMP32,WCEIMP42,WCEIMP52,WCEIMUS2,WCESTP11,WCESTP21,WCESTP31,...,crudeStocksP2E,crudeStocksP9,feddStockRunsP1,feedstockRunsP2,feedstockRunsP3,feedstockRunsP4,feedstockRunsP5,feedstockRunsP9,feedstockRunsUS,grossRunsP9
0,2014-12-26,588.0,2041.0,3040.0,254.0,1139.0,7061.0,12545.0,100552.0,176983.0,...,69759.0,292942.0,9.0,15.0,312.0,2.0,107.0,329.0,445.0,13008.0
1,2015-01-02,700.0,2492.0,2602.0,329.0,731.0,6856.0,11813.0,102885.0,172676.0,...,70787.0,291270.0,5.0,15.0,134.0,16.0,140.0,165.0,309.0,12987.0
2,2015-01-09,876.0,2265.0,2894.0,246.0,1211.0,7492.0,12485.0,105172.0,173585.0,...,71298.0,294200.0,1.0,4.0,180.0,0.0,146.0,184.0,332.0,12572.0
3,2015-01-16,434.0,2172.0,3055.0,344.0,1214.0,7218.0,12542.0,112457.0,174839.0,...,75674.0,302959.0,1.0,3.0,194.0,2.0,142.0,199.0,341.0,11799.0
4,2015-01-23,739.0,2181.0,3314.0,284.0,904.0,7422.0,12875.0,113538.0,180347.0,...,74670.0,309670.0,1.0,8.0,286.0,3.0,137.0,297.0,434.0,12083.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491,2024-05-24,853.0,3020.0,1386.0,411.0,1099.0,6769.0,9109.0,119165.0,258217.0,...,84611.0,401808.0,3.0,26.0,199.0,-4.0,70.0,221.0,294.0,14249.0
492,2024-05-31,393.0,3305.0,1577.0,413.0,1371.0,7058.0,8260.0,117361.0,260563.0,...,81953.0,401852.0,3.0,12.0,342.0,-2.0,84.0,352.0,440.0,14468.0
493,2024-06-07,698.0,3102.0,1986.0,546.0,1972.0,8304.0,7968.0,114942.0,266673.0,...,81127.0,405492.0,3.0,5.0,362.0,-2.0,97.0,365.0,464.0,14378.0
494,2024-06-14,772.0,2882.0,1658.0,394.0,1349.0,7054.0,8374.0,117258.0,262256.0,...,83136.0,403677.0,22.0,12.0,259.0,-4.0,87.0,267.0,378.0,14004.0
